In [1]:
import pandas as pd
import numpy as np
import sys
import os

import mysql.connector
from mysql.connector import Error


def connect_to_database(database, host='localhost', user='root', password='password', v=True):
    try:
        connection = mysql.connector.connect(
            host=host,
            database=database,
            user=user,
            password=password
        )
        
        if connection.is_connected():
            db_info = connection.get_server_info()
            if v:
                print("Connected to MySQL Server version", db_info)
            cursor = connection.cursor()
            cursor.execute("select database();")
            record = cursor.fetchone()
            if v:
                print("You're connected to database:", record)
            return connection, cursor

    except Error as e:
        if v: 
            print("Error while connecting to MySQL", e)

    return None

def close_connection(connection, cursor):
    if cursor:
        cursor.close()
    if connection and connection.is_connected():
        connection.close()
        print("MySQL connection is closed")

def execute_query(connection, cursor, query, params=None):
    try:
        cursor.execute(query, params)
        if query.strip().lower().startswith('select') or query.strip().lower().startswith('show'):
            result = cursor.fetchall()
            return result
        else:
            connection.commit()
            return cursor.rowcount
    except Error as e:
        print("Error executing query:", e)
        return None

def put_res(results, query_type="SELECT"):
    if not results:
        print("No results found.")
        return

    if query_type.lower() in ["select", "show"]:
        for row in results:
            print(row)
    else:
        print(f"Affected rows: {results}")

def fetch_data_to_dataframe(connection, query):
    cursor = None
    try:
        cursor = connection.cursor()
        cursor.execute(query)
        res = cursor.fetchall()
        columns = [desc[0] for desc in cursor.description]
        df = pd.DataFrame(res, columns=columns)
        return df
    except Error as e:
        print(f"Error fetching data: {e}")
        return None
    finally:
        if cursor:
            cursor.close()





In [2]:
con, cur = connect_to_database('flight_prediction')

Connected to MySQL Server version 8.3.0
You're connected to database: ('flight_prediction',)


In [3]:
query = '''
SELECT 
    ID,
    date,
    dep_time,
    arr_time,
    distance
FROM 
    flight
'''
df_flight = fetch_data_to_dataframe(con, query)



In [4]:
query = '''
SELECT 
    ID,
    arr_delay
FROM 
    delay
'''
df_delay = fetch_data_to_dataframe(con, query)

In [5]:
df = pd.merge(df_delay, df_flight, on='ID', how='inner')
df

,ID,arr_delay,date,dep_time,arr_time,distance
0,1,-6.0,2016-01-06,1100,1438,986
1,2,-12.0,2016-01-07,1100,1438,986
2,3,7.0,2016-01-08,1100,1438,986
3,4,-5.0,2016-01-09,1100,1438,986
4,5,113.0,2016-01-10,1100,1438,986
...,...,...,...,...,...,...
5635962,5635963,-13.0,2016-12-31,755,830,451
5635963,5635964,9.0,2016-12-31,1320,1355,451
5635964,5635965,-30.0,2016-12-31,705,1125,1440
5635965,5635966,-4.0,2016-12-31,1220,1235,368
